In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

base_model = "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForSequenceClassification.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16)

model = model.eval()

In [11]:
tokens = tokenizer(["Fuck this economy. I hate aig and their non loan given asses"], return_tensors='pt',
                           padding=True)
output = model(**tokens)

output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
output.argmax()

tensor(1)

In [13]:
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd

def map_output(x):
    dic = {0:4, 1:0, 2:2}
    return dic[x]

def test_sentiment140(model, tokenizer, batch_size=8):
    dataset = load_dataset('sentiment140')
    dataset = dataset["test"]
    dataset = dataset.to_pandas()

    negative_df = dataset.query("sentiment == 0")[:50]
    neutral_df = dataset.query("sentiment == 2")[:50]
    positive_df = dataset.query("sentiment == 4")[:50]

    dataset = pd.concat([negative_df, neutral_df, positive_df])

    dataset = dataset.rename(columns={"sentiment": "target"})


    dataset = dataset[['text', 'target']]

    # print example
    print(f"\n\nPrompt example:\n{dataset['text'][1]}\n\n")

    context = dataset['text'].tolist()
    total_steps = dataset.shape[0]//batch_size + 1
    print(
        f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text = []

    for i in tqdm(range(total_steps)):
        tmp_context = context[i * batch_size:(i+1) * batch_size]

        tokenizer.pad_token = "[PAD]"

        tokens = tokenizer(tmp_context, return_tensors='pt',
                           padding=True)

        output = model(**tokens)
        output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
        out_text.append(output.detach().numpy())
        torch.cuda.empty_cache()

    out_text = [item for sublist in out_text for item in sublist]
    dataset["out_text"] = out_text
    dataset["new_out"] = dataset["out_text"].apply(np.argmax).apply(map_output)

    acc = accuracy_score(dataset["target"], dataset["new_out"])
    f1_macro = f1_score(dataset["target"], dataset["new_out"], average="macro")
    f1_micro = f1_score(dataset["target"], dataset["new_out"], average="micro")
    f1_weighted = f1_score(
        dataset["target"], dataset["new_out"], average="weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    return dataset

dataset = test_sentiment140(model, tokenizer)
dataset



Prompt example:
Reading my kindle2...  Love it... Lee childs is good read.


Total len: 150. Batchsize: 8. Total steps: 19


100%|██████████| 19/19 [00:00<00:00, 20.01it/s]

Acc: 0.41333333333333333. F1 macro: 0.33498970364401465. F1 micro: 0.41333333333333333. F1 weighted (BloombergGPT): 0.3349897036440147. 


,text,target,out_text,new_out
6,Fuck this economy. I hate aig and their non lo...,0,"[0.01448599, 0.8686428, 0.11687122]",0
11,@Karoli I firmly believe that Obama/Pelosi hav...,0,"[0.06383914, 0.5745758, 0.36158502]",0
14,"dear nike, stop with the flywire. that shit is...",0,"[0.055363473, 0.6379475, 0.30668896]",0
16,I was talking to this guy last night and he wa...,0,"[0.11617255, 0.73315835, 0.15066914]",0
18,"@ludajuice Lebron is a Beast, but I'm still ch...",0,"[0.13449314, 0.6116509, 0.25385597]",0
...,...,...,...,...
106,"@psychemedia I really liked @kswedberg's ""Lear...",4,"[0.13677105, 0.1926428, 0.6705861]",2
108,"Very Interesting Ad from Adobe by Goodby, Silv...",4,"[0.17924158, 0.3924566, 0.42830187]",2
109,Goodby Silverstein agency new site! http://www...,4,"[0.146174, 0.25471786, 0.59910816]",2
110,"RT @designplay Goodby, Silverstein's new site:...",4,"[0.2250193, 0.3803487, 0.394632]",2


In [14]:
path = '../../results/general/finbert.csv'

In [15]:
dataset.to_csv(path)

In [16]:
import pandas as pd

df = pd.read_csv(path, index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.696, Recall: 0.413, F1: 0.335, Accuracy: 0.413
